# Representatividade de pedestres em regiões comp ao total

Caderno Jupyter referente à **Issue #5**: *Mapa coroplético de origens e destinos de pedestres com filtros e granularidades* 

**CONTÉM TODOS OS MUNICÍPIOS**

Representatividade de viagens de pedestres por região(zona OD, distrito e município) comparado com o total

---

In [1]:
import pandas as pd
import geopandas
import folium

Carregando datasets

In [2]:
od23_path = "../../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

od_districts_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
od_districts = geopandas.read_file(od_districts_shapefile_path)

od_counties_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Municipios_2023.shp" 
od_counties = geopandas.read_file(od_counties_shapefile_path)

od_zones_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp" 
od_zones = geopandas.read_file(od_zones_shapefile_path)

Convertendo CRS

In [3]:
od_districts = od_districts.to_crs("EPSG:4326")
od_counties = od_counties.to_crs("EPSG:4326")
od_zones = od_zones.to_crs("EPSG:4326")

Filtrando pedestres

In [4]:
pedestrians_walk = od.query('MODOPRIN == 17') ## 17: pedestrians

## Cálculo representatividade (%)

- Por zona (ZONA_O)
- Por município (MUNI_O)

- Por distrito

In [5]:
# total de viagens
total_viagens = pedestrians_walk["FE_VIA"].sum()

# viagens por zona od
zone_share = (
    pedestrians_walk.groupby("ZONA_O")["FE_VIA"].sum()
    .reset_index()
    .rename(columns={"ZONA_O": "NumeroZona", "FE_VIA": "VIAGENS"})
)
# calcula representatividade de viagens de pedestres por zona od
zone_share["PERC"] = 100 * zone_share["VIAGENS"] / total_viagens

# ordena decrescentemente por percentual
zone_share = zone_share.sort_values("PERC", ascending=False)

# viagens por município
county_share = (
    pedestrians_walk.groupby("MUNI_O")["FE_VIA"].sum()
    .reset_index()
    .rename(columns={"MUNI_O": "NumeroMuni", "FE_VIA": "VIAGENS"})
)

# calcula representatividade de viagens de pedestres por município
county_share["PERC"] = 100 * county_share["VIAGENS"] / total_viagens

# ordena decrescentemente por percentual
county_share = county_share.sort_values("PERC", ascending=False)

In [6]:
# viagens por distrito
# 1) junta zonas com distritos
zone_district = od_zones[["NumeroZona", "NumDistrit", "NomeDistri"]].merge(
    zone_share, on="NumeroZona", how="left"
).fillna({"VIAGENS": 0, "PERC": 0})

# 2) agrega por distrito
district_share = (
    zone_district.groupby(["NumDistrit", "NomeDistri"])["VIAGENS"].sum()
    .reset_index()
)

# calcula representatividade de viagens de pedestres por distrito
district_share["PERC"] = 100 * district_share["VIAGENS"] / total_viagens

# ordena decrescentemente por percentual
district_share = district_share.sort_values("PERC", ascending=False)

### Juntar com geometrias

In [7]:
# zonas
zones_map = od_zones.merge(zone_share, on="NumeroZona", how="left").fillna(0)

# municípios
counties_map = od_counties.merge(county_share, on="NumeroMuni", how="left").fillna(0)

# distritos
districts_map = od_districts.merge(district_share, left_on="NumeroDist", right_on="NumDistrit", how="left").fillna(0)

## Mapa

In [8]:
# centro aproximado (SP)
m = folium.Map(location=[-23.55, -46.63], zoom_start=10)

Camada Zona OD

In [9]:
# camada por zona
folium.Choropleth(
    geo_data=zones_map.to_json(),
    name="Zonas OD",
    data=zones_map,
    columns=["NumeroZona", "PERC"],
    key_on="feature.properties.NumeroZona",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Viagens de pedestres (%) - Zona OD"
).add_to(m)

# adiciona tooltip + popup para zonas
folium.GeoJson(
    zones_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"}, 
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeZona", "PERC"],
        aliases=["Zona OD:", "Representatividade (%)"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeZona", "VIAGENS", "PERC"],
        aliases=["Zona OD:", "Viagens:", "Representatividade (%)"],
        localize=True
    ),
    name="zonas od - representatividade",
    show=True
).add_to(m)

Camada Municipio

In [10]:
# camada por município
folium.Choropleth(
    geo_data=counties_map.to_json(),
    name="Municípios",
    data=counties_map,
    columns=["NumeroMuni", "PERC"],
    key_on="feature.properties.NumeroMuni",
    bins=[0, 0.5, 1, 2, 7, 55],
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name="Viagens de pedestres (%) - Municípios",
    show=False
).add_to(m)

# adiciona tooltip + popup para municípios
folium.GeoJson(
    counties_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"},
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeMunici", "PERC"],
        aliases=["Município:", "Representatividade (%)"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeMunici", "VIAGENS", "PERC"],
        aliases=["Município:", "Viagens:", "Representatividade (%)"],
        localize=True
    ),
    name="municipio - representatividade",
    show=False
).add_to(m)

In [11]:
# camada por distrito
folium.Choropleth(
    geo_data=districts_map.to_json(),
    name="Distritos",
    data=districts_map,
    columns=["NumeroDist", "PERC"],
    key_on="feature.properties.NumeroDist",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Viagens de pedestres (%) - Distritos",
    show=False
).add_to(m)

# adiciona tooltip + popup para distritos
folium.GeoJson(
    districts_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"},
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeDistri_x", "PERC"],
        aliases=["Distrito:", "Representatividade (%)"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeDistri_x", "VIAGENS", "PERC"],
        aliases=["Distrito:", "Viagens:", "Representatividade (%)"],
        localize=True
    ),
    name="distrito - representatividade",
    show=False
).add_to(m)

Salvar mapa

In [ ]:
# adicionar controles
folium.LayerControl().add_to(m)

m.save("../built_maps/choropleth_ped_trips_maps/ped_trips.html")